# HIPAA & GDPR Compliance with LakeLogic

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/lakelogic/LakeLogic/blob/main/examples/04_compliance_governance/hipaa_pii_masking/tutorial_hipaa_gdpr_compliance.ipynb) 
[![GitHub Repo](https://img.shields.io/badge/GitHub-Repo-blue?logo=github)](https://github.com/lakelogic/LakeLogic/blob/main/examples/04_compliance_governance/hipaa_pii_masking/tutorial_hipaa_gdpr_compliance.ipynb)


## Business Scenario

A global healthcare and e-commerce company operates across the **United States** and **European Union**. They ingest two types of sensitive data:

1. **US Patient Records** — containing **Protected Health Information (PHI)** under **HIPAA** (names, SSNs, diagnoses)
2. **EU Customer Records** — containing **Personally Identifiable Information (PII)** under **GDPR** (names, emails, phone numbers, consent status)

Both regulations require strict controls, but they differ in important ways:

| Requirement | HIPAA (US) | GDPR (EU) |
| :--- | :--- | :--- |
| **Scope** | Protected Health Information (PHI) | All personal data of EU/EEA residents |
| **Consent** | Implied for treatment | Must be explicit and recorded (Art. 7) |
| **Lawful Basis** | Not required per record | Must be one of six types (Art. 6) |
| **Right to Erasure** | Limited HIPAA exceptions | Full right to be forgotten (Art. 17) |
| **Data Retention** | Minimum 6 years | Storage limitation required (Art. 5(1)(e)) |
| **PII Masking** | De-identification of 18 PHI identifiers | Pseudonymisation or anonymisation |

## Value Proposition

- **Multi-Regulation Compliance**: Apply HIPAA and GDPR policy packs to different tables using the same LakeLogic framework.
- **Automated PII Protection**: Mask sensitive fields during ingestion, not as a downstream afterthought.
- **Audit-Ready Quarantine**: Every failed record is quarantined with the exact rule violation — ready for compliance review.
- **Portable Governance**: Same YAML contracts work on Polars (dev), Spark (prod), DuckDB (CI/CD).

---
# Part 1: HIPAA Compliance (US Healthcare)
---

## 1.1 Setup — Load HIPAA Policy Pack

LakeLogic **Policy Packs** act as organisational baselines. The HIPAA pack enforces SSN format, email validation, and strict schema evolution — even if individual contracts don't specify these rules.

In [ ]:
import os
import yaml
import polars as pl
from pathlib import Path
from lakelogic import DataProcessor

# Resolve paths
def resolve_example_dir(name: str) -> Path:
    cwd = Path.cwd()
    for base in [cwd] + list(cwd.parents):
        candidates = [
            base / name,
            base / "examples" / "04_compliance_governance" / name,
            base / "lakelogic" / "examples" / "04_compliance_governance" / name,
        ]
        for candidate in candidates:
            if candidate.exists():
                return candidate
    return cwd / name


def resolve_repo_root(start: Path) -> Path:
    for base in [start] + list(start.parents):
        if (base / "policy_packs" / "hipaa_compliance.yaml").exists():
            return base
    return start


base_path = resolve_example_dir("hipaa_pii_masking")
repo_root = resolve_repo_root(base_path)

print(f"Example dir: {base_path}")
print(f"Repo root:   {repo_root}")

In [ ]:
# Load and inspect the HIPAA Policy Pack
hipaa_pack_path = repo_root / "policy_packs" / "hipaa_compliance.yaml"

with open(hipaa_pack_path, "r") as f:
    hipaa_pack = yaml.safe_load(f)

print("📋 HIPAA Policy Pack Rules:")
print(f"   Framework: {hipaa_pack.get('defaults', {}).get('metadata', {}).get('compliance_framework', 'N/A')}")
print(f"   Schema Policy: {hipaa_pack.get('defaults', {}).get('schema_policy', {}).get('evolution', 'N/A')}")
print()
for rule in hipaa_pack.get('defaults', {}).get('quality', {}).get('row_rules', []):
    print(f"   🔒 {rule.get('name')}: {rule.get('description')}")

## 1.2 Inspect the Source Data (Raw PHI)

This dataset contains **Protected Health Information** — patient names, Social Security Numbers, and email addresses. Note the intentional violations:
- **Bob** has an invalid email format (`bob@wrong-format`)
- **Edward** has a malformed SSN (`555-555-555` instead of `XXX-XX-XXXX`)

In [ ]:
# Load and preview the raw PHI data
phi_data_path = base_path / "data" / "phi_records.csv"
raw_phi = pl.read_csv(phi_data_path)

print("🏥 Raw Patient Records (contains PHI):")
print(raw_phi)

## 1.3 Run HIPAA Validation with PII Masking

The medical records contract applies two layers of protection:
1. **Quality gates** — SSN format, email validation (from the HIPAA Policy Pack)
2. **PII masking** — Names, SSNs, and emails are replaced with `[PROTECTED]` before reaching Silver

In [ ]:
# Run the HIPAA contract
hipaa_contract = base_path / "contracts" / "medical_records.yaml"
hipaa_processor = DataProcessor(contract=hipaa_contract)

source_df, good_df, bad_df = hipaa_processor.run_source(phi_data_path)

print(f"📊 HIPAA Results:")
print(f"   Source records:      {len(source_df)}")
print(f"   ✅ Valid (Silver):   {len(good_df)}")
print(f"   ❌ Quarantined:     {len(bad_df)}")
print(f"   Reconciliation:     {len(source_df)} = {len(good_df)} + {len(bad_df)} ✓")

## 1.4 Inspect Quarantined Records

Bob and Edward failed the HIPAA policy pack rules and were diverted to quarantine — with the exact failure reason attached for compliance audit.

In [ ]:
print("🚨 QUARANTINE — Failed HIPAA Rules:")
print(bad_df)

## 1.5 Inspect Masked Silver Layer

For the valid records (Alice, Charlie, Diane), notice that `patient_name`, `ssn`, and `email` have been replaced with `[PROTECTED]`. This data is now safe for analytics teams — no raw PHI exposure.

In [ ]:
print("✅ SILVER LAYER — Clean & Masked (HIPAA Safe):")
print(good_df)

---
# Part 2: GDPR Compliance (EU Data Protection) 🇪🇺
---

## 2.1 Load the GDPR Policy Pack

GDPR requires different controls than HIPAA. The key additions:

- **Consent tracking** (Art. 7) — every record must have explicit consent status
- **Lawful basis** (Art. 6) — one of six legal justifications must be documented
- **Storage limitation** (Art. 5(1)(e)) — a retention expiry date must be set
- **Data accuracy** (Art. 5) — email and contact details must be valid

In [ ]:
# Load and inspect the GDPR Policy Pack
gdpr_pack_path = repo_root / "policy_packs" / "gdpr_compliance.yaml"

with open(gdpr_pack_path, "r") as f:
    gdpr_pack = yaml.safe_load(f)

print("📋 GDPR Policy Pack Rules:")
print(f"   Framework: {gdpr_pack.get('defaults', {}).get('metadata', {}).get('compliance_framework', 'N/A')}")
print(f"   Regions:   {gdpr_pack.get('defaults', {}).get('metadata', {}).get('applicable_regions', 'N/A')}")
print(f"   PII Policy: {gdpr_pack.get('defaults', {}).get('metadata', {}).get('pii_policy', 'N/A')}")
print()
for rule in gdpr_pack.get('defaults', {}).get('quality', {}).get('row_rules', []):
    print(f"   🔒 {rule.get('name')}: {rule.get('description')}")

## 2.2 Inspect the EU Customer Data

This dataset contains EU customer records with intentional GDPR violations:

| Customer | Violation | GDPR Article |
| :--- | :--- | :--- |
| **Sofia García** | Invalid email format (`sofia@bad-format`) | Art. 5 (Accuracy) |
| **Lars Eriksson** | Missing retention date, empty phone | Art. 5(1)(e) (Storage Limitation) |
| **Maria Rossi** | Missing consent status (`null`) | Art. 7 (Consent) |
| **Hans Weber** | Invalid lawful basis (`invalid_basis`) | Art. 6 (Lawful Basis) |

In [ ]:
# Load and preview the EU customer data
eu_data_path = base_path / "data" / "eu_customers.csv"
raw_eu = pl.read_csv(eu_data_path)

print("🇪🇺 Raw EU Customer Records (contains PII):")
print(raw_eu)

## 2.3 Run GDPR Validation with PII Masking

The EU customer contract validates:
1. **Consent** — explicitly recorded (`true` or `false`, not `null`)
2. **Lawful basis** — must be one of the six GDPR Art. 6 types
3. **Retention date** — must be set (storage limitation)
4. **Email format** — basic validity check
5. **PII masking** — names, emails, and phones replaced with `[GDPR_REDACTED]`

In [ ]:
# Run the GDPR contract
gdpr_contract = base_path / "contracts" / "eu_customers.yaml"
gdpr_processor = DataProcessor(contract=gdpr_contract)

eu_source, eu_good, eu_bad = gdpr_processor.run_source(eu_data_path)

print(f"📊 GDPR Results:")
print(f"   Source records:      {len(eu_source)}")
print(f"   ✅ Valid (Silver):   {len(eu_good)}")
print(f"   ❌ Quarantined:     {len(eu_bad)}")
print(f"   Reconciliation:     {len(eu_source)} = {len(eu_good)} + {len(eu_bad)} ✓")

## 2.4 Inspect GDPR Quarantine

Each quarantined record includes the specific GDPR article violation, making it audit-ready for the Data Protection Officer (DPO).

In [ ]:
print("🚨 QUARANTINE — Failed GDPR Rules:")
print(eu_bad)

## 2.5 Inspect GDPR-Compliant Silver Layer

Valid records have PII pseudonymised (`[GDPR_REDACTED]`). Notice that:
- `full_name`, `email`, and `phone` are masked
- `consent_given`, `lawful_basis`, and `retention_expires_at` are preserved (needed for compliance tracking)
- Non-PII fields (`country_code`, `id`) remain untouched

In [ ]:
print("✅ SILVER LAYER — GDPR-Compliant & Pseudonymised:")
print(eu_good)

---
# Part 3: Comparison & Summary
---

## 3.1 Side-by-Side Results

In [ ]:
print("╔══════════════════════════════════════════════════════════════╗")
print("║           COMPLIANCE SUMMARY                                ║")
print("╠══════════════════════════════════════════════════════════════╣")
print("║                                                             ║")
print("║  🏥 HIPAA (US Healthcare)                                   ║")
print(f"║     Source: {len(source_df):>3}  |  ✅ Valid: {len(good_df):>3}  |  ❌ Quarantine: {len(bad_df):>3}  ║")
print(f"║     Masking: patient_name, ssn, email → [PROTECTED]        ║")
print("║     Rules: SSN format (XXX-XX-XXXX), email validation       ║")
print("║                                                             ║")
print("║  🇪🇺 GDPR (EU Data Protection)                              ║")
print(f"║     Source: {len(eu_source):>3}  |  ✅ Valid: {len(eu_good):>3}  |  ❌ Quarantine: {len(eu_bad):>3}  ║")
print(f"║     Masking: full_name, email, phone → [GDPR_REDACTED]     ║")
print("║     Rules: consent (Art.7), lawful basis (Art.6),           ║")
print("║             retention (Art.5), email format (Art.5)         ║")
print("║                                                             ║")
print("╠══════════════════════════════════════════════════════════════╣")
print("║  Key Difference:                                            ║")
print("║  HIPAA focuses on data FORMAT (SSN, email patterns)         ║")
print("║  GDPR focuses on data GOVERNANCE (consent, legal basis,     ║")
print("║  retention, and the right to be forgotten)                  ║")
print("║                                                             ║")
print("║  LakeLogic handles BOTH with the same contract framework.   ║")
print("╚══════════════════════════════════════════════════════════════╝")

## 3.2 Key Takeaways

### What LakeLogic Gives You

1. **Policy Packs as Compliance Templates** — HIPAA, GDPR (or SOX, PCI-DSS, etc.) can be defined once and applied across hundreds of tables. The compliance team manages the policy; data engineers just reference it.

2. **Automated PII Masking at Ingestion** — Shift-left security. PII is masked *before* data reaches the Silver layer, not as a downstream afterthought. Analysts never see raw identifiers.

3. **Audit-Ready Quarantine** — Every failed record includes the exact rule name, the GDPR/HIPAA article reference, and the failure reason. This is exactly what regulators and DPOs need during audits.

4. **100% Reconciliation** — `source_count = valid_count + quarantine_count`. No silent data loss. Every record is accounted for.

5. **Engine-Agnostic** — These same contracts run identically on Polars (local dev), Spark (production), or DuckDB (CI/CD). No compliance logic rewrite when changing platforms.

### GDPR-Specific Features

- **Consent tracking** — Validates that consent is explicitly recorded (not assumed)
- **Lawful basis validation** — Ensures one of the six Art. 6 types is documented
- **Storage limitation** — Enforces retention dates, supporting the right to erasure
- **Pseudonymisation** — PII fields replaced with `[GDPR_REDACTED]` in Silver layer
- **Jurisdiction awareness** — Country code validation for regional compliance tracking

### Next Steps

- Add `--ai` to `lakelogic bootstrap` to auto-detect PII fields and suggest masking strategies
- Create custom policy packs for SOX, PCI-DSS, or industry-specific regulations
- Use `lakelogic generate --invalid-ratio 0.15 --ai` to stress-test compliance rules with edge cases